In [1]:
import os
import sys
import pathlib
import platform
from copy import deepcopy

from typing import (
    Dict,
    List,
    Union
)

In [2]:
# CLI
mod_path: str = os.path.join(str(pathlib.Path(os.path.abspath(os.getcwd())).parents[1]))
sys.path.append(mod_path)

In [3]:
from convert_source.cs_utils.fileio import (
    ConversionError,
    Command
)

In [4]:
from convert_source.cs_utils.utils import (
    SubDataInfo,
    list_dict,
    comp_dict,
    depth,
    convert_image_data,
    file_to_screen,
    collect_info,
    get_metadata,
    zeropad
)

In [5]:
from convert_source.batch_convert import (
    BIDSImgData,
    read_config,
    bids_id,
    _gather_bids_name_args,
    _get_bids_name_args,
    make_bids_name,
    data_to_bids,
    batch_proc,
    source_to_bids
)

In [6]:
from convert_source.cs_utils.bids_info import construct_bids_name

In [7]:
from convert_source.cs_utils.const import BIDS_PARAM

In [8]:
scripts_dir: str = os.path.join(os.getcwd(),'helper.scripts')
test_config1: str = os.path.join(scripts_dir,'test.1.config.yml')
test_config2: str = os.path.join(scripts_dir,'test.2.config.yml')

data_dir: str = os.path.join(os.getcwd(),'test.study_dir')
dcm_test_data: str = os.path.join(data_dir,'TEST001-UNIT001','data.dicom','ST000000')

In [9]:
# dcm2niix should already be downloaded
dcm2niix_cmd: Command = Command("dcm2niix")
dcm2niix_cmd.check_dependency(path_envs=[os.getcwd()])

True

In [15]:
verbose: bool = True
[search_dict,
bids_search,
bids_map,
meta_dict,
exclusion_list] = read_config(config_file=test_config2,
                            verbose=verbose)

Initialized parameters from configuration file
Categorizing search terms
Including BIDS related search term settings
Corresponding BIDS mapping settings
Including additional settings for metadata
Exclusion option implemented


In [16]:

subs_data: List[SubDataInfo] = collect_info(parent_dir=data_dir,exclusion_list=exclusion_list)

In [18]:
subs_data

[{'sub': 'TEST001', 'ses': 'UNIT001', 'data': '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.dicom/ST000000/SE000000/MR000002.dcm'},
 {'sub': 'TEST001', 'ses': 'UNIT001', 'data': '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.dicom/ST000000/SE000001/MR000024.dcm'},
 {'sub': 'TEST001', 'ses': 'UNIT001', 'data': '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.dicom/ST000000/SE000002/MR000011.dcm'},
 {'sub': 'TEST001', 'ses': 'UNIT001', 'data': '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.nifti/DWI_68_DIR.nii.gz'},
 {'sub': 'TEST001', 'ses': 'UNIT001', 'data': '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.nifti/DWI_B0.nii.gz'},
 {'sub': 'TEST001', 'ses': 'UNIT001', 'd

In [26]:
for i in range(0,len(subs_data)):
    if '.par' in subs_data[i].data.lower():
        print(f"{i} : {subs_data[i].data}")

In [13]:
# This one throws error
subs_data[0].data

'/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.dicom/ST000000/SE000000/MR000002.dcm'

In [14]:
# dcm test
subs_data[1].data

'/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.dicom/ST000000/SE000001/MR000024.dcm'

In [15]:
# par test - NOTE: no corresponding image data
subs_data[11].data

'/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.parrec/AX_SWIP_MPR.PAR'

In [16]:
# nii tests
subs_data[9].data

'/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.nifti/rsfMRI.nii.gz'

In [17]:
bids_name_dict: Dict = deepcopy(BIDS_PARAM)
bids_name_dict['info']['sub'] = subs_data[9].sub
if subs_data[9].ses:
    bids_name_dict['info']['ses'] = subs_data[9].ses

[bids_name_dict, 
 modality_type, 
 modality_label, 
 task] = bids_id(s=subs_data[9].data,
                 search_dict=search_dict,
                 bids_search=bids_search,
                 bids_map=bids_map,
                 bids_name_dict=bids_name_dict,
                 parent_dir=data_dir)
print(modality_type)
print(modality_label)
print(task)

func
bold
rest


In [18]:
[meta_com_dict, 
meta_scan_dict] = get_metadata(dictionary=meta_dict,
                            modality_type=modality_type,
                            task=task)

In [19]:
meta_com_dict

{'Manufacturer': 'Vendor',
 'ManufacturersModelName': 'Scanner model',
 'MagneticFieldStrength': 3,
 'InstitutionName': 'Institution Name'}

In [20]:
meta_scan_dict

{'ParallelAcquisitionTechnique': 'GRAPPA',
 'PhaseEncodingDirection': 'j',
 'MultibandAccelerationFactor': 2,
 'TaskName': 'Resting State',
 'NumberOfVolumesDiscardedByScanner': 5}

In [10]:
out_dir: str = os.path.join(os.getcwd(),'test.bids')

In [11]:
out_dir

'/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids'

In [23]:
[imgs,
jsons,
bvals,
bvecs] = data_to_bids(sub_data=subs_data[9],
                    bids_name_dict=bids_name_dict,
                    out_dir=out_dir,
                    modality_type=modality_type,
                    modality_label=modality_label,
                    task=task,
                    meta_dict=meta_com_dict,
                    mod_dict=meta_scan_dict)

In [24]:
imgs

['/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/func/sub-TEST001_ses-UNIT001_task-rest_run-01_bold.nii.gz']

In [25]:
# source_to_bids(sub_data=subs_data[1],
#                     bids_name_dict=bids_name_dict,
#                     out_dir=out_dir,
#                     modality_type=modality_type,
#                     modality_label=modality_label,
#                     task=task,
#                     meta_dict=meta_com_dict,
#                     mod_dict=meta_scan_dict)

In [42]:
subs_bids = BIDSImgData

In [44]:
len(subs_bids.imgs)

0

In [45]:
subs_bids = BIDSImgData(imgs,
jsons,
bvals,
bvecs)

In [46]:
subs_bids.imgs

['/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/anat/sub-TEST001_ses-UNIT001_run-01_T2w.nii.gz',
 '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/anat/sub-TEST001_ses-UNIT001_run-02_T2w.nii.gz']

In [47]:
# subs_bids.add(imgs)

In [48]:
imgs

['/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/anat/sub-TEST001_ses-UNIT001_run-05_T2w.nii.gz',
 '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/anat/sub-TEST001_ses-UNIT001_run-06_T2w.nii.gz']

In [49]:
subs_bids.jsons

['/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/anat/sub-TEST001_ses-UNIT001_run-01_T2w.json',
 '']

In [50]:
len(subs_bids.bvals)

2

In [51]:
subs_bids.add(imgs=imgs,
            jsons=jsons,
            bvals=bvals,
            bvecs=bvecs)

(['/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/anat/sub-TEST001_ses-UNIT001_run-05_T2w.nii.gz',
  '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/anat/sub-TEST001_ses-UNIT001_run-06_T2w.nii.gz',
  '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/anat/sub-TEST001_ses-UNIT001_run-05_T2w.nii.gz',
  '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/anat/sub-TEST001_ses-UNIT001_run-06_T2w.nii.gz'],
 ['/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/anat/sub-TEST001_ses-UNIT001_run-05_T2w.json',
  '',
  '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/anat/sub-TEST001_ses-UNIT001_run-05_T2w.json',
  ''],
 ['', '', '', ''],
 ['',

In [55]:
len(subs_bids.bvecs)

4

In [30]:
bids_name_dict = construct_bids_name(subs_data[9],modality_type,modality_label,task=task,out_dir=out_dir,zero_pad=2)

In [31]:
make_bids_name(bids_name_dict,modality_type)

('sub-TEST001_ses-UNIT001_task-rest_run-01_bold',
 'sub-TEST001_ses-UNIT001_task-rest_run-02_bold',
 'sub-TEST001_ses-UNIT001_task-rest_run-03_bold',
 'sub-TEST001_ses-UNIT001_task-rest_run-04_bold')

In [32]:
bids_name_dict

{'info': {'sub': 'TEST001', 'ses': 'UNIT001'},
 'anat': {'acq': '', 'ce': '', 'rec': '', 'run': '', 'modality_label': ''},
 'func': {'task': 'rest',
  'acq': '',
  'ce': '',
  'dir': '',
  'rec': '',
  'run': '01',
  'echo': '',
  'modality_label': 'bold'},
 'dwi': {'acq': '', 'dir': '', 'run': '', 'modality_label': ''},
 'fmap': {'acq': '',
  'run': '',
  'case1': {'phasediff': '', 'magnitude1': '', 'magnitude2': ''},
  'case2': {'phase1': '', 'phase2': '', 'magnitude1': '', 'magnitude2': ''},
  'case3': {'magnitude': '', 'fieldmap': ''},
  'case4': {'ce': '', 'dir': '', 'modality_label': 'epi'}}}

In [12]:
[imgs,
jsons,
bvals,
bvecs] = batch_proc(config_file=test_config2,
        study_img_dir=data_dir,
        out_dir=out_dir,
        verbose=True)

Initialized parameters from configuration file
Categorizing search terms
Including BIDS related search term settings
Corresponding BIDS mapping settings
Including additional settings for metadata
Exclusion option implemented
/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/tmp_dir_7874
/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/tmp_dir_6911
ERROR: Error: Missing images. Found 8 images, expected 27 slices per volume and instance number (0020,0013) ranges from 2 to 25

/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/tmp_dir_4787
ERROR: Error: Missing images. Found 9 images, expected 32 slices per volume and instance number (0020,0013) ranges from 1 to 16



In [13]:
len(imgs)

11

In [14]:
len(jsons)

11

In [19]:
imgs

['',
 '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/anat/sub-TEST001_ses-UNIT001_run-01_T2w.nii.gz',
 '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/anat/sub-TEST001_ses-UNIT001_run-02_T2w.nii.gz',
 '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/func/sub-TEST001_ses-UNIT001_task-rest_run-01_sbref.nii.gz',
 '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/func/sub-TEST001_ses-UNIT001_task-rest_run-02_sbref.nii.gz',
 '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/dwi/sub-TEST001_ses-UNIT001_acq-b800b1000b1500b2000b3000_run-01_sbref.nii.gz',
 '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/dwi/sub-TEST001_ses-UNIT001_ac